<a href="https://colab.research.google.com/github/archivesunleashed/notebooks/blob/master/parquet_pyspark_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with Archives Unleashed Parquet Derivatives

In this notebook, we'll setup an enviroment, then download a dataset of web archive collection derivatives that were produced with the [Archives Unleashed Toolkit](https://github.com/archivesunleashed/aut/). These derivatives are in the [Apache Parquet](https://parquet.apache.org/) format, which is a [columnar storage](http://en.wikipedia.org/wiki/Column-oriented_DBMS) format. These derivatives and generally small enough to work with on your local machine, and can be easily converted to Pandas DataFrames as demostrated below.

**[Binary Analysis](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#binary-analysis)**
- [Audio](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-audio-information)
- [Images](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-image-information)
- [PDFs](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-pdf-information)
- [Presentation program files](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-presentation-program-files-information)
- [Spreadsheets](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-spreadsheet-information)
- [Text files](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-text-files-information)
- [Videos](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-video-information)
- [Word processor files](https://github.com/archivesunleashed/aut-docs/blob/master/current/binary-analysis.md#extract-word-processor-files-information)

**Web Pages**

`.webpages()` 

Produces a DataFrame with the following columns:
  - `crawl_date`
  - `url`
  - `mime_type_web_server`
  - `mime_type_tika`
  - `content`

**Web Graph**

`.webgraph()` 

Produces a DataFrame with the following columns:
  - `crawl_date`
  - `src`
  - `dest`
  - `anchor`

**Image Links**

`.imageLinks()`

Produces a DataFrame with the following columns:
  - `src`
  - `image_url`

**Domaisn**

`.webpages().groupBy(ExtractDomainDF($"url").alias("url")).count().sort($"count".desc)`

Produces a DataFrame with the following columns:
- domain
- count

# Setup Dependencies

First we'll need to setup Java, Apache Spark, and PySpark bindings

In [0]:
%%capture

!apt-get update
!apt-get install -y openjdk-8-jdk-headless -qq 
!apt-get install maven -qq

!curl -L "https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz" > spark-2.4.4-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

# Dataset

We will need a web archive dataset to work with.

The one we'll use in this example notebook comes from [Bibliothèque et Archives nationales du Québec](https://www.banq.qc.ca/accueil/). It is a web archive collection of the Ministry of Environment of Québec (2011-2014), that has been  processed by the [Archives Unleashed Toolkit](https://github.com/archivesunleashed/aut/). Merci beaucoup banq!

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3599771.svg)](https://doi.org/10.5281/zenodo.3599771)

Curious about the size the derivative Parquet output compared to the size of the web archive collection?

The total size of all 12 Parquet deriatives is 594M, with `webpages` being the largest (528M) since it has a column with full text (`content`).

```
2.5M	./videos
344K	./domains
1.7M	./word-processor-files
24K	./presentation-program-files
1.7M	./spreadsheets
880K	./audio
4.4M	./images
528M	./webpages
1.7M	./text-files
3.9M	./pdfs
29M	./webgraph
22M	./imagelinks
594M	.
```

The total size of the web archive collection is 165G.

In [0]:
%%capture

!curl -L "https://zenodo.org/record/3598450/files/environnement-qc.tar.gz?download=1" > environment-qc-parquet.tar.gz
!tar -xzf environment-qc-parquet.tar.gz

In [3]:
!ls -1 parquet

audio
domains
imagelinks
images
pdfs
presentation-program-files
spreadsheets
text-files
videos
webgraph
webpages
word-processor-files


# Environment

Next, we'll setup our environment so we can work with the Parquet output in PySpark.

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# Loading our Archives Unleashed Datasets as DataFrames

Next, we'll load up our datasets to work with, and show a preview of each.

In [6]:
audio = spark.read.parquet("parquet/audio")
audio.show()

+--------------------+--------------------+---------+--------------------+--------------+--------------------+
|                 url|            filename|extension|mime_type_web_server|mime_type_tika|                 md5|
+--------------------+--------------------+---------+--------------------+--------------+--------------------+
|http://www.mddep....|connaissances-eau...|      mp3|          audio/mpeg|    audio/mpeg|f1f9bd2570b2eef6b...|
|http://www.ragedu...| operation-raton.wmv|      wma|      video/x-ms-wmv|audio/x-ms-wma|4e5763994a906b6aa...|
|http://www.ragedu...| operation-raton.wmv|      wma|      video/x-ms-wmv|audio/x-ms-wma|4e5763994a906b6aa...|
|http://www.ragedu...| operation-raton.wmv|      wma|      video/x-ms-wmv|audio/x-ms-wma|4e5763994a906b6aa...|
|http://www.ragedu...| operation-raton.wmv|      wma|      video/x-ms-wmv|audio/x-ms-wma|4e5763994a906b6aa...|
|http://www.deficl...|110314%20Steven%2...|      mp3|          audio/mpeg|    audio/mpeg|b6722993460210738...|
|

In [7]:
domains =  spark.read.parquet("parquet/domains")
domains.show()

+--------------------+-----+
|                 url|count|
+--------------------+-----+
|     www.addthis.com|    1|
|www.rechargeonsla...|    1|
|  static.addinto.com|    1|
|     code.google.com|    1|
|www.senv.mddep.go...|    1|
|     swdlp.apple.com|    1|
|       www.google.ca|    1|
|       get.adobe.com|    1|
|www.paiement.mdde...|    1|
|     support.pqm.net|    1|
|     www.addinto.com|    1|
|feeds.iucnredlist...|    1|
| www.cbcq.gouv.qc.ca|    1|
|        www.undp.org|    1|
|    l.addthiscdn.com|    1|
| fls.doubleclick.net|    1|
|         twitter.com|    1|
|search.microsoft.com|    1|
|www.espacej.gouv....|    1|
|    webcasts.pqm.net|    1|
+--------------------+-----+
only showing top 20 rows



In [8]:
images = spark.read.parquet("parquet/images")
images.show()

+--------------------+--------------------+---------+--------------------+--------------+-----+------+--------------------+
|                 url|            filename|extension|mime_type_web_server|mime_type_tika|width|height|                 md5|
+--------------------+--------------------+---------+--------------------+--------------+-----+------+--------------------+
|http://www.mddep....|rencontre-chasseu...|      jpg|          image/jpeg|    image/jpeg|  200|   150|1f2b23e9a20f5c4bb...|
|http://www.mddep....|rencontre-chasseu...|      jpg|          image/jpeg|    image/jpeg|  200|   150|1f2b23e9a20f5c4bb...|
|http://mddep.gouv...|rencontre-chasseu...|      jpg|          image/jpeg|    image/jpeg|  200|   150|1f2b23e9a20f5c4bb...|
|http://www.mddefp...|rencontre-chasseu...|      jpg|          image/jpeg|    image/jpeg|  200|   150|1f2b23e9a20f5c4bb...|
|http://www.mddep....|rencontre-chasseu...|      jpg|          image/jpeg|    image/jpeg|  200|   150|1f2b23e9a20f5c4bb...|
|http://

In [10]:
network = spark.read.parquet("parquet/webgraph")
network.show()

+----------+--------------------+--------------------+--------------------+
|crawl_date|                 src|                dest|              anchor|
+----------+--------------------+--------------------+--------------------+
|  20190502|http://mddefp.gou...|https://www.quebe...|            Ministre|
|  20190502|http://mddefp.gou...|https://geoinonda...|Geoinondations.go...|
|  20190502|http://mddefp.gou...|https://cse.googl...|           Recherche|
|  20190502|http://mddefp.gou...|http://www.quebec...|                    |
|  20190502|http://mddefp.gou...|http://www.droita...|� Gouvernement du...|
|  20190502|http://mddefp.gou...|https://www.quebe...|            Ministre|
|  20190502|http://mddefp.gou...|http://www.mddelc...|l'affiche officielle|
|  20190502|http://mddefp.gou...|http://www.mddelc...|    bonnes pratiques|
|  20190502|http://mddefp.gou...|http://www.mddelc...|programme Environ...|
|  20190502|http://mddefp.gou...|https://cse.googl...|           Recherche|
|  20190502|

In [12]:
pdfs = spark.read.parquet("parquet/pdfs")
pdfs.show()

+--------------------+------------------+---------+--------------------+---------------+--------------------+
|                 url|          filename|extension|mime_type_web_server| mime_type_tika|                 md5|
+--------------------+------------------+---------+--------------------+---------------+--------------------+
|http://mddefp.gou...|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://www.mddefp...|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://www.mddep....|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://mddep.gouv...|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://www.mddep....|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://mddefp.gou...|   Map-Zone-26.pdf|      pdf|     application/pdf|application/pdf|34eb18168e9ca934c...|
|http://ww

In [13]:
presentation_program = spark.read.parquet("parquet/presentation-program-files")
presentation_program.show()

+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|                 url|            filename|extension|mime_type_web_server|      mime_type_tika|                 md5|
+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|http://www.enviro...|           JEAP.pptx|     pptx|application/vnd.o...|application/vnd.o...|c3cf8193a17007d48...|
|http://mddep.gouv...|phosphore-abitibi...|      ppt|application/vnd.m...|application/vnd.m...|f355c8d364bbd7c69...|
|http://mddefp.gou...|phosphore-abitibi...|      ppt|application/vnd.m...|application/vnd.m...|f355c8d364bbd7c69...|
|http://www.mddep....|phosphore-abitibi...|      ppt|application/vnd.m...|application/vnd.m...|f355c8d364bbd7c69...|
|http://mddep.gouv...|phosphore-abitibi...|      ppt|application/vnd.m...|application/vnd.m...|f355c8d364bbd7c69...|
|http://www.mddelc...|phosphore-abitibi...|      ppt|application

In [14]:
spreadsheets = spark.read.parquet("parquet/spreadsheets")
spreadsheets.show()

+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|                 url|            filename|extension|mime_type_web_server|      mime_type_tika|                 md5|
+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|http://mddefp.gou...|liste-etablisseme...|     xlsx|application/vnd.o...|application/vnd.o...|2aa0b7c4f741a13ac...|
|http://mddefp.gou...|30-3-2-adaptation...|     xlsm|application/vnd.m...|application/vnd.m...|b962130c2b1c169fe...|
|http://mddefp.gou...|17-1-ecocamionnag...|     xlsx|application/vnd.o...|application/vnd.o...|5dc7c1910bcd8af6c...|
|http://mddefp.gou...|9-3-financement-m...|     xlsm|application/vnd.m...|application/vnd.m...|35259010efa4b4a7d...|
|http://mddefp.gou...|    14-7-PIEVAL.xlsm|     xlsm|application/vnd.m...|application/vnd.m...|3520b8cdafd7c500e...|
|http://mddefp.gou...|ge-4-gestion-inte...|     xlsm|application

In [15]:
videos = spark.read.parquet("parquet/videos")
videos.show()

+--------------------+-------------+---------+--------------------+--------------+--------------------+
|                 url|     filename|extension|mime_type_web_server|mime_type_tika|                 md5|
+--------------------+-------------+---------+--------------------+--------------+--------------------+
|http://r2---sn-9p...|videoplayback|      mp4|           video/mp4|     video/mp4|d628294e7df1c7b12...|
|https://r5---sn-t...|videoplayback|      mp4|           video/mp4|     video/mp4|d610d5763cc3d1a5a...|
|http://r1---sn-9p...|videoplayback|      mp4|           video/mp4|     video/mp4|d5e37c1c79441e40b...|
|http://r2---sn-9p...|videoplayback|      mp4|           video/mp4|     video/mp4|d5a21cd8ee5bccc31...|
|http://r2---sn-9p...|videoplayback|      mp4|           video/mp4|     video/mp4|d5559807a134ff6f4...|
|http://r3---sn-9p...|videoplayback|      mp4|           video/mp4|     video/mp4|d499c661c34549771...|
|http://r3---sn-hu...|videoplayback|      mp4|           video/m

In [16]:
word_processor = spark.read.parquet("parquet/word-processor-files")
word_processor.show()

+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|                 url|            filename|extension|mime_type_web_server|      mime_type_tika|                 md5|
+--------------------+--------------------+---------+--------------------+--------------------+--------------------+
|http://mddefp.gou...|     formulaire.docx|     docx|application/vnd.o...|application/vnd.o...|49437b65e78fc8d07...|
|http://www.mddep....|3-aide_particulie...|      doc|  application/msword|  application/msword|4943475a254e3b927...|
|http://www.mddep....|         annexe5.doc|      doc|  application/msword|  application/msword|491bdb25f3ee1358b...|
|http://www.mddefp...|         annexe5.doc|      doc|  application/msword|  application/msword|491bdb25f3ee1358b...|
|http://mddefp.gou...|Attestation-perso...|      doc|  application/msword|  application/msword|4868aa8e259881533...|
|http://www.mddep....|Attestation-perso...|      doc|  applicati

In [17]:
text_files = spark.read.parquet("parquet/text-files")
text_files.show()

+--------------------+--------------------+---------+--------------------+--------------+--------------------+
|                 url|            filename|extension|mime_type_web_server|mime_type_tika|                 md5|
+--------------------+--------------------+---------+--------------------+--------------+--------------------+
|http://r1---sn-9p...|       videoplayback|         |          text/plain|           N/A|d41d8cd98f00b204e...|
|http://whqlibdoc....|       repository.pl|       pl|          text/plain|           N/A|d41d8cd98f00b204e...|
|http://r1---sn-9p...|       videoplayback|         |          text/plain|           N/A|d41d8cd98f00b204e...|
|https://r5---sn-t...|       videoplayback|         |          text/plain|           N/A|d41d8cd98f00b204e...|
|https://r4---sn-t...|       videoplayback|         |          text/plain|           N/A|d41d8cd98f00b204e...|
|https://r1---sn-t...|       videoplayback|         |          text/plain|           N/A|d41d8cd98f00b204e...|
|

# Data Analysis

Now that we have all of our datasets loaded up, we can begin to work with it!


## Counting total files, and unique files

Count number of rows (how many images are in the web archive collection).

In [18]:
images.count()

156166

How many unique images are in the collection?

In [19]:
from pyspark.sql.functions import countDistinct
images.select(countDistinct("md5")).show()

+-------------------+
|count(DISTINCT md5)|
+-------------------+
|              18287|
+-------------------+



What are the top 10 most occuring filenames in the collection?

In [21]:
import pyspark.sql.functions as func
images.groupBy('filename').count().orderBy('count', ascending=False).show(10)

+-------------+-----+
|     filename|count|
+-------------+-----+
|  carte-p.jpg| 1196|
|   carte2.jpg|  924|
|   carte1.jpg|  875|
|  carte-g.jpg|  660|
|    carte.jpg|  576|
| carte-an.jpg|  575|
| carte-qc.jpg|  575|
|  carte-G.jpg|  484|
|  carte_p.jpg|  473|
|carte_web.jpg|  431|
+-------------+-----+
only showing top 10 rows



What are the top 10 most occuring images in the collection?

In [22]:
import pyspark.sql.functions as func
images.groupBy('md5').count().orderBy('count', ascending=False).show(10, False)

+--------------------------------+-----+
|md5                             |count|
+--------------------------------+-----+
|5283d313972a24f0e71c47ae3c99958b|192  |
|b09dc3225d5e1377c52c06feddc33bfe|192  |
|a4d3ddfb1a95e87650c624660d67765a|192  |
|e7d1f7750c16bc835bf1cfe1bf322d46|192  |
|89663337857f6d769fbcaed7278cc925|77   |
|497db34fffa0e278f57ae614b4b758a0|64   |
|58e5d8676dfcc4205551314d98fb2624|61   |
|100322cfd242ee75dd5a744526f08d6b|56   |
|7252e42a951b5e449ea02c517839ed6d|53   |
|65274f9eaa4c585b7c35193ebb04e0d7|53   |
+--------------------------------+-----+
only showing top 10 rows



What's the information around all of the occurances of `5283d313972a24f0e71c47ae3c99958b`?

In [23]:
images.filter("md5 = '5283d313972a24f0e71c47ae3c99958b'").show(192, False)

+---------------------------------------------------------------+----------+---------+--------------------+--------------+-----+------+--------------------------------+
|url                                                            |filename  |extension|mime_type_web_server|mime_type_tika|width|height|md5                             |
+---------------------------------------------------------------+----------+---------+--------------------+--------------+-----+------+--------------------------------+
|http://www.mddefp.gouv.qc.ca//poissons/st-charles/tumeur.jpg   |tumeur.jpg|jpg      |image/jpeg          |image/jpeg    |310  |220   |5283d313972a24f0e71c47ae3c99958b|
|http://www.mddep.gouv.qc.ca/poissons/yamaska/tumeur.jpg        |tumeur.jpg|jpg      |image/jpeg          |image/jpeg    |310  |220   |5283d313972a24f0e71c47ae3c99958b|
|http://mddefp.gouv.qc.ca/poissons/st-maurice/tumeur.jpg        |tumeur.jpg|jpg      |image/jpeg          |image/jpeg    |310  |220   |5283d313972a24f0e71c